In [173]:
import os
import re
import sklearn
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.metrics import *
from sklearn.linear_model import *
from sklearn.model_selection import *

pd.set_option('display.max_columns', None)

In [174]:
all_games_df = pd.read_csv('all_games_df.csv')
all_games_df = all_games_df[['is_tourney','Season','HTeamID','RTeamID','Hwin']]
all_games_df.head()

,is_tourney,Season,HTeamID,RTeamID,Hwin
0,0.0,2004,1113.0,1168.0,1
1,0.0,2004,1337.0,1168.0,0
2,0.0,2004,1414.0,1168.0,1
3,0.0,2004,1142.0,1168.0,1
4,0.0,2004,1429.0,1168.0,1


In [175]:
all_games_df.tail()

,is_tourney,Season,HTeamID,RTeamID,Hwin
71322,0.0,2021,1259.0,1248.0,1
71323,0.0,2021,1259.0,1248.0,1
71324,0.0,2021,1259.0,1248.0,1
71325,0.0,2021,1259.0,1248.0,1
71326,0.0,2021,1259.0,1248.0,1


In [176]:
kp_df = pd.read_csv('kp_all.csv')
kp_df.sort_index()

,team,conf,adjem,adjo,adjd,luck,Season,TeamID
0,gonzaga,WCC,36.62,126.1,89.5,0.018,2021,1211.0
1,michigan,B10,30.79,119.4,88.6,0.026,2021,1276.0
2,illinois,B10,30.61,119.0,88.3,0.030,2021,1228.0
3,baylor,B12,29.50,123.3,93.8,0.050,2021,1124.0
4,iowa,B10,28.90,123.5,94.6,-0.028,2021,1234.0
...,...,...,...,...,...,...,...,...
6862,nicholls st.,Slnd,-26.22,87.1,113.3,-0.070,2002,1311.0
6863,arkansas pine bluff,SWAC,-28.85,82.2,111.1,-0.014,2002,NaN
6864,mount st. mary's,NEC,-29.36,85.1,114.5,0.019,2002,1291.0
6865,morris brown,ind,-29.95,85.1,115.1,0.044,2002,1289.0


In [177]:
regseason_df = pd.read_csv('regseason_df.csv')
regseason_df  = regseason_df[['TeamID','Season','wins_top25','PointMargin','FG','FG3']]
regseason_df.head()

,TeamID,Season,wins_top25,PointMargin,FG,FG3
0,1101,2014,0,-15.476190,0.410184,0.380275
1,1101,2015,0,-10.857143,0.407501,0.381507
2,1101,2016,0,-6.407407,0.442606,0.364517
3,1101,2017,0,-4.760000,0.457867,0.371026
4,1101,2018,0,-2.148148,0.452366,0.327240


In [178]:
regseason_df.tail()

,TeamID,Season,wins_top25,PointMargin,FG,FG3
6529,1467,2021,0,-0.388889,0.448842,0.313038
6530,1468,2021,0,4.277778,0.500646,0.366353
6531,1469,2021,0,-10.578947,0.403965,0.285279
6532,1470,2021,0,-4.266667,0.432487,0.361324
6533,1471,2021,0,-7.928571,0.459440,0.372476


In [179]:
massey_df = pd.read_csv('C:/Users/FLUXNATURE/Desktop/New Kaggle world/NCAAM_stage_two_two/MDataFiles_Stage2/MMasseyOrdinals.csv')
POM_df = massey_df[massey_df['SystemName'].str.contains("POM")]
POM_end_df = POM_df.loc[POM_df['RankingDayNum'] == 128]
POM_end_df.rename(columns={'OrdinalRank': 'RankPOM'}, inplace=True)
POM_end_df = POM_end_df[['Season','TeamID','RankPOM']]
POM_end_df.head()

C:\Users\FLUXNATURE\Anaconda3\envs\tf\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Season,TeamID,RankPOM
392714,2005,1102,54
392715,2005,1103,75
392716,2005,1104,18
392717,2005,1105,291
392718,2005,1106,300


In [180]:
POM_end_df.tail()

,Season,TeamID,RankPOM
4317029,2021,1467,273
4317030,2021,1468,177
4317031,2021,1469,323
4317032,2021,1470,262
4317033,2021,1471,231


Create test data set

In [181]:
#Test set (this sets the data up in the format Kaggle needs for scoring)
df_seeds = pd.read_csv('C:/Users/FLUXNATURE/Desktop/New Kaggle world/NCAAM_stage_two_two/MDataFiles_Stage2/MNCAATourneySeeds.csv')
df_seeds = df_seeds[df_seeds['Season']==2021]
df_19_tourney = df_seeds.merge(df_seeds, how='inner', on='Season')
df_19_tourney = df_19_tourney[df_19_tourney['TeamID_x'] < df_19_tourney['TeamID_y']]
df_19_tourney['ID'] = df_19_tourney['Season'].astype(str) + '_' \
              + df_19_tourney['TeamID_x'].astype(str) + '_' \
              + df_19_tourney['TeamID_y'].astype(str)
df_19_tourney['SeedInt_x'] = [int(x[1:3]) for x in df_19_tourney['Seed_x']]
df_19_tourney['SeedInt_y'] = [int(x[1:3]) for x in df_19_tourney['Seed_y']]

#Make home team lower seed (consistent with training data)
df_19_tourney.loc[df_19_tourney['SeedInt_x']<df_19_tourney['SeedInt_y'], 'HTeamID'] = df_19_tourney['TeamID_x']
df_19_tourney.loc[df_19_tourney['SeedInt_x']>df_19_tourney['SeedInt_y'], 'HTeamID'] = df_19_tourney['TeamID_y']
df_19_tourney.loc[df_19_tourney['SeedInt_x']<df_19_tourney['SeedInt_y'], 'RTeamID'] = df_19_tourney['TeamID_y']
df_19_tourney.loc[df_19_tourney['SeedInt_x']>df_19_tourney['SeedInt_y'], 'RTeamID'] = df_19_tourney['TeamID_x']
df_19_tourney.loc[df_19_tourney['SeedInt_x']==df_19_tourney['SeedInt_y'], 'HTeamID'] = df_19_tourney['TeamID_x']
df_19_tourney.loc[df_19_tourney['SeedInt_x']==df_19_tourney['SeedInt_y'], 'RTeamID'] = df_19_tourney['TeamID_y']

df_19_tourney.loc[df_19_tourney['SeedInt_x']<df_19_tourney['SeedInt_y'], 'HSeed'] = df_19_tourney['SeedInt_x']
df_19_tourney.loc[df_19_tourney['SeedInt_x']>df_19_tourney['SeedInt_y'], 'HSeed'] = df_19_tourney['SeedInt_y']
df_19_tourney.loc[df_19_tourney['SeedInt_x']<df_19_tourney['SeedInt_y'], 'RSeed'] = df_19_tourney['SeedInt_y']
df_19_tourney.loc[df_19_tourney['SeedInt_x']>df_19_tourney['SeedInt_y'], 'RSeed'] = df_19_tourney['SeedInt_x']
df_19_tourney.loc[df_19_tourney['SeedInt_x']==df_19_tourney['SeedInt_y'], 'HSeed'] = df_19_tourney['SeedInt_x']
df_19_tourney.loc[df_19_tourney['SeedInt_x']==df_19_tourney['SeedInt_y'], 'RSeed'] = df_19_tourney['SeedInt_y']

df_19_tourney['is_tourney'] = 1

df_19_tourney = df_19_tourney.drop(['Seed_x','Seed_y','TeamID_x','TeamID_y','SeedInt_x','SeedInt_y'], axis=1)
df_19_tourney.sort_index()


,Season,ID,HTeamID,RTeamID,HSeed,RSeed,is_tourney
2,2021,2021_1276_1400,1276.0,1400.0,1.0,3.0,1
8,2021,2021_1276_1382,1276.0,1382.0,1.0,9.0,1
10,2021,2021_1276_1277,1276.0,1277.0,1.0,11.0,1
11,2021,2021_1276_1417,1276.0,1417.0,1.0,11.0,1
13,2021,2021_1276_1422,1276.0,1422.0,1.0,13.0,1
...,...,...,...,...,...,...,...
4617,2021,2021_1216_1439,1439.0,1216.0,10.0,16.0,1
4618,2021,2021_1216_1429,1429.0,1216.0,11.0,16.0,1
4619,2021,2021_1216_1457,1457.0,1216.0,12.0,16.0,1
4620,2021,2021_1216_1317,1317.0,1216.0,13.0,16.0,1


In [182]:
home_road = ['H','R']
for hr in home_road:
    df_19_tourney = pd.merge(df_19_tourney, regseason_df, left_on=['Season',hr+'TeamID'], right_on = ['Season','TeamID'], how='left')
    df_19_tourney.rename(columns={'wins_top25': hr+'wins_top25'}, inplace=True)
    df_19_tourney.rename(columns={'PointMargin': hr+'PointMargin'}, inplace=True)
    df_19_tourney.rename(columns={'FG': hr+'FG'}, inplace=True)
    df_19_tourney.rename(columns={'FG3': hr+'FG3'}, inplace=True)
    df_19_tourney = df_19_tourney.drop(['TeamID'], axis=1)

In [183]:
df_19_tourney.tail()

,Season,ID,HTeamID,RTeamID,HSeed,RSeed,is_tourney,Hwins_top25,HPointMargin,HFG,HFG3,Rwins_top25,RPointMargin,RFG,RFG3
2273,2021,2021_1216_1439,1439.0,1216.0,10.0,16.0,1,8,6.619048,0.452007,0.357875,0,2.608696,0.436148,0.329668
2274,2021,2021_1216_1429,1429.0,1216.0,11.0,16.0,1,8,10.481481,0.442059,0.335270,0,2.608696,0.436148,0.329668
2275,2021,2021_1216_1457,1457.0,1216.0,12.0,16.0,1,0,12.750000,0.462443,0.350717,0,2.608696,0.436148,0.329668
2276,2021,2021_1216_1317,1317.0,1216.0,13.0,16.0,1,0,7.640000,0.470076,0.369846,0,2.608696,0.436148,0.329668
2277,2021,2021_1216_1331,1331.0,1216.0,15.0,16.0,1,0,1.826087,0.450063,0.391579,0,2.608696,0.436148,0.329668


In [184]:
for hr in home_road:
    df_19_tourney = pd.merge(df_19_tourney, POM_end_df, left_on=['Season',hr+'TeamID'], right_on = ['Season','TeamID'], how='left')
    df_19_tourney.rename(columns={'RankPOM': hr+'RankPOM'}, inplace=True)
    df_19_tourney = df_19_tourney.drop(['TeamID'], axis=1)

In [185]:
df_19_tourney.tail()

,Season,ID,HTeamID,RTeamID,HSeed,RSeed,is_tourney,Hwins_top25,HPointMargin,HFG,HFG3,Rwins_top25,RPointMargin,RFG,RFG3,HRankPOM,RRankPOM
2273,2021,2021_1216_1439,1439.0,1216.0,10.0,16.0,1,8,6.619048,0.452007,0.357875,0,2.608696,0.436148,0.329668,47,197
2274,2021,2021_1216_1429,1429.0,1216.0,11.0,16.0,1,8,10.481481,0.442059,0.335270,0,2.608696,0.436148,0.329668,45,197
2275,2021,2021_1216_1457,1457.0,1216.0,12.0,16.0,1,0,12.750000,0.462443,0.350717,0,2.608696,0.436148,0.329668,86,197
2276,2021,2021_1216_1317,1317.0,1216.0,13.0,16.0,1,0,7.640000,0.470076,0.369846,0,2.608696,0.436148,0.329668,80,197
2277,2021,2021_1216_1331,1331.0,1216.0,15.0,16.0,1,0,1.826087,0.450063,0.391579,0,2.608696,0.436148,0.329668,154,197


In [186]:
efficiency_list = ['conf','adjem','adjo','adjd','luck','TeamID']
for hr in home_road:
    df_19_tourney = pd.merge(df_19_tourney, kp_df, left_on=[hr+'TeamID','Season'], right_on = ['TeamID','Season'], how='left')
    df_19_tourney = df_19_tourney.drop(['TeamID'], axis=1)
    for metric in efficiency_list:
        df_19_tourney.rename(columns={metric: hr+metric}, inplace=True)
    if hr == 'H':
        df_19_tourney.rename(columns={'team': 'home'}, inplace=True)
    if hr == 'R':
        df_19_tourney.rename(columns={'team': 'road'}, inplace=True)

In [187]:
df_19_tourney.tail()

,Season,ID,HTeamID,RTeamID,HSeed,RSeed,is_tourney,Hwins_top25,HPointMargin,HFG,HFG3,Rwins_top25,RPointMargin,RFG,RFG3,HRankPOM,RRankPOM,home,Hconf,Hadjem,Hadjo,Hadjd,Hluck,road,Rconf,Radjem,Radjo,Radjd,Rluck
2273,2021,2021_1216_1439,1439.0,1216.0,10.0,16.0,1,8,6.619048,0.452007,0.357875,0,2.608696,0.436148,0.329668,47,197,virginia tech,ACC,15.22,110.0,94.8,0.066,hartford,AE,-2.73,97.5,100.2,0.078
2274,2021,2021_1216_1429,1429.0,1216.0,11.0,16.0,1,8,10.481481,0.442059,0.335270,0,2.608696,0.436148,0.329668,45,197,utah st.,MWC,16.51,105.7,89.2,-0.032,hartford,AE,-2.73,97.5,100.2,0.078
2275,2021,2021_1216_1457,1457.0,1216.0,12.0,16.0,1,0,12.750000,0.462443,0.350717,0,2.608696,0.436148,0.329668,86,197,winthrop,BSth,8.94,105.1,96.2,0.032,hartford,AE,-2.73,97.5,100.2,0.078
2276,2021,2021_1216_1317,1317.0,1216.0,13.0,16.0,1,0,7.640000,0.470076,0.369846,0,2.608696,0.436148,0.329668,80,197,north texas,CUSA,11.70,105.4,93.7,-0.042,hartford,AE,-2.73,97.5,100.2,0.078
2277,2021,2021_1216_1331,1331.0,1216.0,15.0,16.0,1,0,1.826087,0.450063,0.391579,0,2.608696,0.436148,0.329668,154,197,oral roberts,Sum,1.22,108.7,107.4,0.013,hartford,AE,-2.73,97.5,100.2,0.078


In [188]:
df_19_tourney['Htourny20plus'] = 0
df_19_tourney['Rtourny20plus'] = 0

experienced_teams = ['kansas','north carolina','kentucky','duke','michigan st.','wisconsin','florida','villanova','gonzaga','louisville','arizona','xavier','connecticut','syracuse','butler','ohio st.','ucla','west virginia','texas','michigan','pittsburgh','memphis','oregon']
for team in experienced_teams:
    df_19_tourney.loc[df_19_tourney['home']==team, 'Htourny20plus'] = 1
    df_19_tourney.loc[df_19_tourney['road']==team, 'Rtourny20plus'] = 1

In [189]:
df_19_tourney.tail()

,Season,ID,HTeamID,RTeamID,HSeed,RSeed,is_tourney,Hwins_top25,HPointMargin,HFG,HFG3,Rwins_top25,RPointMargin,RFG,RFG3,HRankPOM,RRankPOM,home,Hconf,Hadjem,Hadjo,Hadjd,Hluck,road,Rconf,Radjem,Radjo,Radjd,Rluck,Htourny20plus,Rtourny20plus
2273,2021,2021_1216_1439,1439.0,1216.0,10.0,16.0,1,8,6.619048,0.452007,0.357875,0,2.608696,0.436148,0.329668,47,197,virginia tech,ACC,15.22,110.0,94.8,0.066,hartford,AE,-2.73,97.5,100.2,0.078,0,0
2274,2021,2021_1216_1429,1429.0,1216.0,11.0,16.0,1,8,10.481481,0.442059,0.335270,0,2.608696,0.436148,0.329668,45,197,utah st.,MWC,16.51,105.7,89.2,-0.032,hartford,AE,-2.73,97.5,100.2,0.078,0,0
2275,2021,2021_1216_1457,1457.0,1216.0,12.0,16.0,1,0,12.750000,0.462443,0.350717,0,2.608696,0.436148,0.329668,86,197,winthrop,BSth,8.94,105.1,96.2,0.032,hartford,AE,-2.73,97.5,100.2,0.078,0,0
2276,2021,2021_1216_1317,1317.0,1216.0,13.0,16.0,1,0,7.640000,0.470076,0.369846,0,2.608696,0.436148,0.329668,80,197,north texas,CUSA,11.70,105.4,93.7,-0.042,hartford,AE,-2.73,97.5,100.2,0.078,0,0
2277,2021,2021_1216_1331,1331.0,1216.0,15.0,16.0,1,0,1.826087,0.450063,0.391579,0,2.608696,0.436148,0.329668,154,197,oral roberts,Sum,1.22,108.7,107.4,0.013,hartford,AE,-2.73,97.5,100.2,0.078,0,0


In [190]:
df_19_tourney['HBig4Conf'] = 0
df_19_tourney['RBig4Conf'] = 0
conferences = ['ACC','B10','B12','SEC', 'BE']
for conf in conferences:
    df_19_tourney.loc[df_19_tourney['Hconf']==conf, 'HBig4Conf'] = 1
    df_19_tourney.loc[df_19_tourney['Rconf']==conf, 'RBig4Conf'] = 1

In [191]:
list(df_19_tourney)

['Season',
 'ID',
 'HTeamID',
 'RTeamID',
 'HSeed',
 'RSeed',
 'is_tourney',
 'Hwins_top25',
 'HPointMargin',
 'HFG',
 'HFG3',
 'Rwins_top25',
 'RPointMargin',
 'RFG',
 'RFG3',
 'HRankPOM',
 'RRankPOM',
 'home',
 'Hconf',
 'Hadjem',
 'Hadjo',
 'Hadjd',
 'Hluck',
 'road',
 'Rconf',
 'Radjem',
 'Radjo',
 'Radjd',
 'Rluck',
 'Htourny20plus',
 'Rtourny20plus',
 'HBig4Conf',
 'RBig4Conf']

In [192]:
df_19_tourney.head()

,Season,ID,HTeamID,RTeamID,HSeed,RSeed,is_tourney,Hwins_top25,HPointMargin,HFG,HFG3,Rwins_top25,RPointMargin,RFG,RFG3,HRankPOM,RRankPOM,home,Hconf,Hadjem,Hadjo,Hadjd,Hluck,road,Rconf,Radjem,Radjo,Radjd,Rluck,Htourny20plus,Rtourny20plus,HBig4Conf,RBig4Conf
0,2021,2021_1276_1400,1276.0,1400.0,1.0,3.0,1,20,10.875,0.482426,0.386504,14,6.423077,0.457855,0.361382,3,26,michigan,B10,30.79,119.4,88.6,0.026,texas,B12,20.58,113.8,93.3,0.017,1,1,1,1
1,2021,2021_1276_1382,1276.0,1382.0,1.0,9.0,1,20,10.875,0.482426,0.386504,0,10.150000,0.449645,0.345861,3,27,michigan,B10,30.79,119.4,88.6,0.026,st. bonaventure,A10,20.82,111.3,90.5,0.014,1,0,1,0
2,2021,2021_1276_1277,1276.0,1277.0,1.0,11.0,1,20,10.875,0.482426,0.386504,12,-1.518519,0.424548,0.322040,3,55,michigan,B10,30.79,119.4,88.6,0.026,michigan st.,B10,14.01,107.0,93.0,0.106,1,1,1,1
3,2021,2021_1276_1417,1276.0,1417.0,1.0,11.0,1,20,10.875,0.482426,0.386504,4,4.346154,0.463615,0.378662,3,42,michigan,B10,30.79,119.4,88.6,0.026,ucla,P12,15.92,113.4,97.5,0.024,1,1,1,0
4,2021,2021_1276_1422,1276.0,1422.0,1.0,13.0,1,20,10.875,0.482426,0.386504,0,6.172414,0.424032,0.302067,3,98,michigan,B10,30.79,119.4,88.6,0.026,unc greensboro,SC,8.15,104.1,96.0,0.005,1,0,1,0


In [193]:
df_19_tourney.tail()

,Season,ID,HTeamID,RTeamID,HSeed,RSeed,is_tourney,Hwins_top25,HPointMargin,HFG,HFG3,Rwins_top25,RPointMargin,RFG,RFG3,HRankPOM,RRankPOM,home,Hconf,Hadjem,Hadjo,Hadjd,Hluck,road,Rconf,Radjem,Radjo,Radjd,Rluck,Htourny20plus,Rtourny20plus,HBig4Conf,RBig4Conf
2273,2021,2021_1216_1439,1439.0,1216.0,10.0,16.0,1,8,6.619048,0.452007,0.357875,0,2.608696,0.436148,0.329668,47,197,virginia tech,ACC,15.22,110.0,94.8,0.066,hartford,AE,-2.73,97.5,100.2,0.078,0,0,1,0
2274,2021,2021_1216_1429,1429.0,1216.0,11.0,16.0,1,8,10.481481,0.442059,0.335270,0,2.608696,0.436148,0.329668,45,197,utah st.,MWC,16.51,105.7,89.2,-0.032,hartford,AE,-2.73,97.5,100.2,0.078,0,0,0,0
2275,2021,2021_1216_1457,1457.0,1216.0,12.0,16.0,1,0,12.750000,0.462443,0.350717,0,2.608696,0.436148,0.329668,86,197,winthrop,BSth,8.94,105.1,96.2,0.032,hartford,AE,-2.73,97.5,100.2,0.078,0,0,0,0
2276,2021,2021_1216_1317,1317.0,1216.0,13.0,16.0,1,0,7.640000,0.470076,0.369846,0,2.608696,0.436148,0.329668,80,197,north texas,CUSA,11.70,105.4,93.7,-0.042,hartford,AE,-2.73,97.5,100.2,0.078,0,0,0,0
2277,2021,2021_1216_1331,1331.0,1216.0,15.0,16.0,1,0,1.826087,0.450063,0.391579,0,2.608696,0.436148,0.329668,154,197,oral roberts,Sum,1.22,108.7,107.4,0.013,hartford,AE,-2.73,97.5,100.2,0.078,0,0,0,0


In [194]:
df_19_tourney = df_19_tourney.fillna(df_19_tourney.mean())

In [195]:
df_19_tourney.to_csv('test_combos_df_2021.csv', index=False)

In [196]:
#submission_3 = pd.read_csv('C:/Users/FLUXNATURE/Desktop/New Kaggle world/NCAAM_stage_two_two/MDataFiles_Stage2/MSampleSubmissionStage2.csv')

In [197]:
#submission_3.head()

,ID,Pred
0,2021_1101_1104,0.5
1,2021_1101_1111,0.5
2,2021_1101_1116,0.5
3,2021_1101_1124,0.5
4,2021_1101_1140,0.5


In [198]:
#submission_3.tail()

,ID,Pred
2273,2021_1452_1457,0.5
2274,2021_1452_1458,0.5
2275,2021_1455_1457,0.5
2276,2021_1455_1458,0.5
2277,2021_1457_1458,0.5
